<a href="https://colab.research.google.com/github/Justdead12/CO2-Emissions-End-Assignment/blob/main/CO2_Emissions_End_Assignment_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [238]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%reload_ext google.colab.data_table

df = pd.read_csv('https://raw.githubusercontent.com/Justdead12/CO2-Emissions-End-Assignment/main/levelized-cost-of-energy.csv')

In [239]:
# Hier maak ik de dataframe bruikbaar.

df = df.dropna(how='all')
df = df[(df['Entity'] == 'World')]
df = df.drop(columns= ['Code', 'Entity', 'CSP LCOE (2019 USD/kWh)'])
df['Year'] = df['Year'].round(0)
df = df.set_index('Year')

# Dataframe klaar maken voor de eerste berekening.
# Hier zorg ik ervoor dat we alleen de rijen te zien krijgen na 2009. De informatie in de jaren ervoor vond ik minder belangrijk, want hier waren alleen gegevens bekent van de onshore wind.
# Vervolgens zorg ik ervoor de de enige NaN wordt opgevuld voor een getal dat tussen die omliggende getallen ligt.

df1 = df[(df.index > 2009)]
df1 = df1.sort_values('Year', ascending= False)
df1['Geothermal LCOE (2019 USD/kWh)'] = df1['Geothermal LCOE (2019 USD/kWh)'].interpolate(method= 'linear')

In [240]:
# Ik heb hier een functie gemaakt die aan de hand van gegevens die ingevoerd worden, ervoor zorgt dat de linear regression eruit komt.
# Deze laat ik doorlopen in de komende 11 jaar. Tot en met 2030.
# In de dataframe is vervolgens te zien welke van de energybronnen (als ze hetzelfde reageren als het verleden), in 2030 het goedkoopste is.
# In dit geval is dat solar energy, hier zou je zelfs geld op toe krijgen.

def add_column_regression(column, regression_column, column_number):
    average_array = np.polyfit(df1.index, column, 1)
    average = np.poly1d(average_array)
    df1.insert(column_number, regression_column, average(df1.index))
    energy_source = df1[[regression_column]]
    energy_source = energy_source.reset_index()
    energy_source = energy_source.sort_values('Year', ascending= True)
    average_change = energy_source.iloc[1, 1] - energy_source.iloc[0, 1]
    linear_line = [(2020, (energy_source.iloc[9, 1] + average_change)),
                   (2021, (energy_source.iloc[9, 1] + (average_change * 2))),
                   (2022, (energy_source.iloc[9, 1] + (average_change * 3))),
                   (2023, (energy_source.iloc[9, 1] + (average_change * 4))),
                   (2024, (energy_source.iloc[9, 1] + (average_change * 5))),
                   (2025, (energy_source.iloc[9, 1] + (average_change * 6))),
                   (2026, (energy_source.iloc[9, 1] + (average_change * 7))),
                   (2027, (energy_source.iloc[9, 1] + (average_change * 8))),
                   (2028, (energy_source.iloc[9, 1] + (average_change * 9))),
                   (2029, (energy_source.iloc[9, 1] + (average_change * 10))),
                   (2030, (energy_source.iloc[9, 1] + (average_change * 11)))]
    df1_future = pd.DataFrame(linear_line,
                             columns=['Year', regression_column])
    energy_source = energy_source.append(df1_future)
    return energy_source  

hydro = (add_column_regression(df1['Hydro LCOE (2019 USD/kWh)'], 'Hydro R', 1))
solar = (add_column_regression(df1['Solar LCOE (2019 USD/kWh)'], 'Solar R', 3))
onshore_wind = (add_column_regression(df1['Onshore wind LCOE (2019 USD/kWh)'], 'Onshore Wind R', 5))
bioenergy = (add_column_regression(df1['Bioenergy LCOE (2019 USD/kWh)'], 'Bioenergy R', 7))
geothermal = (add_column_regression(df1['Geothermal LCOE (2019 USD/kWh)'], 'Geothermal R', 9))
offshore_wind = (add_column_regression(df1['Offshore wind LCOE (2019 USD/kWh)'], 'Offshore Wind R', 11))

df2 = pd.DataFrame(index=df.index)
df2 = df2.reset_index()
df2 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(hydro, solar), onshore_wind), bioenergy), geothermal), offshore_wind)

df2

,Year,Hydro R,Solar R,Onshore Wind R,Bioenergy R,Geothermal R,Offshore Wind R
0,2010,0.036352,0.310022,0.088964,0.071135,0.061611,0.180309
1,2011,0.037949,0.279030,0.085194,0.070769,0.062840,0.174418
2,2012,0.039545,0.248039,0.081424,0.070403,0.064068,0.168527
3,2013,0.041142,0.217047,0.077655,0.070038,0.065297,0.162636
4,2014,0.042738,0.186056,0.073885,0.069672,0.066526,0.156745
5,2015,0.044335,0.155064,0.070115,0.069307,0.067754,0.150855
6,2016,0.045931,0.124073,0.066345,0.068941,0.068983,0.144964
7,2017,0.047528,0.093081,0.062576,0.068576,0.070211,0.139073
8,2018,0.049125,0.062090,0.058806,0.068210,0.071440,0.133182
9,2019,0.050721,0.031098,0.055036,0.067844,0.072668,0.127291


In [241]:
# Aangezien de solar energy prijs enorm is gedaalt in de eerste 2 a 3 jaar, ben ik hieronder gaan kijken wanneer we kijken vanaf 2013.
# Zelfs dan is er te zien dat je met solar energy geld toe krijgt en zelfs de offshore wind.
df3 = df[(df.index > 2012)]

def add_column_regression(column, regression_column, column_number):
    average_array = np.polyfit(df3.index, column, 1)
    average = np.poly1d(average_array)
    df3.insert(column_number, regression_column, average(df3.index))
    energy_source = df3[[regression_column]]
    energy_source = energy_source.reset_index()
    energy_source = energy_source.sort_values('Year', ascending= True)
    average_change = energy_source.iloc[1, 1] - energy_source.iloc[0, 1]
    linear_line = [(2020, (energy_source.iloc[6, 1] + average_change)),
                   (2021, (energy_source.iloc[6, 1] + (average_change * 2))),
                   (2022, (energy_source.iloc[6, 1] + (average_change * 3))),
                   (2023, (energy_source.iloc[6, 1] + (average_change * 4))),
                   (2024, (energy_source.iloc[6, 1] + (average_change * 5))),
                   (2025, (energy_source.iloc[6, 1] + (average_change * 6))),
                   (2026, (energy_source.iloc[6, 1] + (average_change * 7))),
                   (2027, (energy_source.iloc[6, 1] + (average_change * 8))),
                   (2028, (energy_source.iloc[6, 1] + (average_change * 9))),
                   (2029, (energy_source.iloc[6, 1] + (average_change * 10))),
                   (2030, (energy_source.iloc[6, 1] + (average_change * 11)))]
    df3_future = pd.DataFrame(linear_line,
                             columns=['Year', regression_column])
    energy_source = energy_source.append(df3_future)
    return energy_source  

hydro = (add_column_regression(df3['Hydro LCOE (2019 USD/kWh)'], 'Hydro R', 1))
solar = (add_column_regression(df3['Solar LCOE (2019 USD/kWh)'], 'Solar R', 3))
onshore_wind = (add_column_regression(df3['Onshore wind LCOE (2019 USD/kWh)'], 'Onshore Wind R', 5))
bioenergy = (add_column_regression(df3['Bioenergy LCOE (2019 USD/kWh)'], 'Bioenergy R', 7))
geothermal = (add_column_regression(df3['Geothermal LCOE (2019 USD/kWh)'], 'Geothermal R', 9))
offshore_wind = (add_column_regression(df3['Offshore wind LCOE (2019 USD/kWh)'], 'Offshore Wind R', 11))

df4 = pd.DataFrame(index=df.index)
df4 = df4.reset_index()
df4 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(hydro, solar), onshore_wind), bioenergy), geothermal), offshore_wind)

df4

,Year,Hydro R,Solar R,Onshore Wind R,Bioenergy R,Geothermal R,Offshore Wind R
0,2013,0.043175,0.173061,0.080571,0.082392,0.061086,0.185714
1,2014,0.044255,0.154350,0.076000,0.078880,0.063172,0.173714
2,2015,0.045336,0.135639,0.071429,0.075369,0.065259,0.161714
3,2016,0.046417,0.116929,0.066857,0.071858,0.067345,0.149714
4,2017,0.047497,0.098218,0.062286,0.068346,0.069432,0.137714
5,2018,0.048578,0.079507,0.057714,0.064835,0.071518,0.125714
6,2019,0.049659,0.060796,0.053143,0.061324,0.073604,0.113714
7,2020,0.050740,0.042086,0.048571,0.057812,0.075691,0.101714
8,2021,0.051820,0.023375,0.044000,0.054301,0.077777,0.089714
9,2022,0.052901,0.004664,0.039429,0.050790,0.079863,0.077714


In [242]:
# Laten we kijken al we er nog 3 jaar vanaf halen.
# Dan gaat de solar energy nog sneller in de min. 
# Uit alle 3 de berekeningen is dus te zien dat de solar energy de beste price voor de consument heeft in de toekomst.
df5 = df[(df.index > 2015)]

def add_column_regression(column, regression_column, column_number):
    average_array = np.polyfit(df5.index, column, 1)
    average = np.poly1d(average_array)
    df5.insert(column_number, regression_column, average(df5.index))
    energy_source = df5[[regression_column]]
    energy_source = energy_source.reset_index()
    energy_source = energy_source.sort_values('Year', ascending= True)
    average_change = energy_source.iloc[1, 1] - energy_source.iloc[0, 1]
    linear_line = [(2020, (energy_source.iloc[3, 1] + average_change)),
                   (2021, (energy_source.iloc[3, 1] + (average_change * 2))),
                   (2022, (energy_source.iloc[3, 1] + (average_change * 3))),
                   (2023, (energy_source.iloc[3, 1] + (average_change * 4))),
                   (2024, (energy_source.iloc[3, 1] + (average_change * 5))),
                   (2025, (energy_source.iloc[3, 1] + (average_change * 6))),
                   (2026, (energy_source.iloc[3, 1] + (average_change * 7))),
                   (2027, (energy_source.iloc[3, 1] + (average_change * 8))),
                   (2028, (energy_source.iloc[3, 1] + (average_change * 9))),
                   (2029, (energy_source.iloc[3, 1] + (average_change * 10))),
                   (2030, (energy_source.iloc[3, 1] + (average_change * 11)))]
    df5_future = pd.DataFrame(linear_line,
                             columns=['Year', regression_column])
    energy_source = energy_source.append(df5_future)
    return energy_source  

hydro = (add_column_regression(df5['Hydro LCOE (2019 USD/kWh)'], 'Hydro R', 1))
solar = (add_column_regression(df5['Solar LCOE (2019 USD/kWh)'], 'Solar R', 3))
onshore_wind = (add_column_regression(df5['Onshore wind LCOE (2019 USD/kWh)'], 'Onshore Wind R', 5))
bioenergy = (add_column_regression(df5['Bioenergy LCOE (2019 USD/kWh)'], 'Bioenergy R', 7))
geothermal = (add_column_regression(df5['Geothermal LCOE (2019 USD/kWh)'], 'Geothermal R', 9))
offshore_wind = (add_column_regression(df5['Offshore wind LCOE (2019 USD/kWh)'], 'Offshore Wind R', 11))

df6 = pd.DataFrame(index=df.index)
df6 = df6.reset_index()
df6 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(hydro, solar), onshore_wind), bioenergy), geothermal), offshore_wind)

df6

,Year,Hydro R,Solar R,Onshore Wind R,Bioenergy R,Geothermal R,Offshore Wind R
0,2016,0.053558,0.11088,0.0670,0.071860,0.071266,0.1443
1,2017,0.051027,0.09586,0.0625,0.068394,0.071456,0.1346
2,2018,0.048496,0.08084,0.0580,0.064928,0.071646,0.1249
3,2019,0.045965,0.06582,0.0535,0.061462,0.071836,0.1152
4,2020,0.043434,0.05080,0.0490,0.057997,0.072026,0.1055
5,2021,0.040903,0.03578,0.0445,0.054531,0.072216,0.0958
6,2022,0.038372,0.02076,0.0400,0.051065,0.072406,0.0861
7,2023,0.035841,0.00574,0.0355,0.047599,0.072595,0.0764
8,2024,0.033310,-0.00928,0.0310,0.044134,0.072785,0.0667
9,2025,0.030779,-0.02430,0.0265,0.040668,0.072975,0.0570
